In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import pandas as pd
import io
from google.colab import files
uploaded = files.upload()

In [ ]:
!pip install -qU \
  "pinecone-client[grpc]" \
  sentence-transformers

In [ ]:
import os
os.listdir (path='.')

In [ ]:
import pandas

In [ ]:
df = pandas.read_json('./yelp_academic_dataset_business.json', lines=True)
df.head()

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print(f"You are using {device}. This is much slower than using "
          "a CUDA-enabled GPU. If on Colab you can change this by "
          "clicking Runtime > Change runtime type > GPU.")

#model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
model = SentenceTransformer('all-mpnet-base-v2', device=device)

model

In [ ]:
import os
import pinecone

pinecone.init(
    api_key='to_be_input',
    environment='to_be_input'
)

In [ ]:
index_name = 'yelpbusiness-search'

if index_name in pinecone.list_indexes ():
  pinecone.delete_index(index_name)

# only create index if it doesn't exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=model.get_sentence_embedding_dimension(),
        metric='cosine'
    )


# now connect to the index
index = pinecone.GRPCIndex(index_name)


In [ ]:
model.get_sentence_embedding_dimension()

In [ ]:
#join all the columns together
df ['descritpon'] = df['name'] + ' with the location of ' + df['address'] + ', in the city of ' + df['city'] + ', in the state of ' + df['state'] + ' with the zip code of ' + df['postal_code'].astype(str) + ' in the business categories of ' + df['categories']
businesslist = df['descritpon'].tolist()

In [ ]:
fullids = df['business_id'].tolist()

In [ ]:
from tqdm.auto import tqdm

batch_size = 256

for i in tqdm(range(0, len(businesslist), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(businesslist))
    # create IDs batch
    ids = fullids[i : i_end]
    # create metadata batch
    metadatas = [{'text': text} for text in businesslist[i:i_end]]
    # create embeddings
    xc = model.encode(businesslist[i:i_end])
    # create records list for upsert
    records = zip(ids, xc, metadatas)
    # upsert to Pinecone
    index.upsert(vectors=records)

# check number of records in the index
index.describe_index_stats()

In [ ]:
!pip install -qU \
  "pinecone-client[grpc]" \
  sentence-transformers

In [ ]:
# get the pipecone client
import os
import pinecone

pinecone.init(
    api_key='to_be_input',
    environment='to_be_input'
)
index_name = 'yelpbusiness-search'
index = pinecone.GRPCIndex(index_name)

In [ ]:
# get the model
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print(f"You are using {device}. This is much slower than using "
          "a CUDA-enabled GPU. If on Colab you can change this by "
          "clicking Runtime > Change runtime type > GPU.")

#model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
model = SentenceTransformer('all-mpnet-base-v2', device=device)

In [ ]:
query = "italian restaurants in tampa?"

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(xq, top_k=20, include_metadata=True)
xc

In [ ]:
query = "italian restaurants in florida?"

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(xq, top_k=20, include_metadata=True)
xc